In [11]:
import pypylon.pylon as py
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [12]:
NUM_CAMERAS = 5

In [13]:
# setup demo environment with 5 cameras
os.environ["PYLON_CAMEMU"] = f"{NUM_CAMERAS}"

tlf = py.TlFactory.GetInstance()

# create a device filter for Pylon CamEmu devices
di = py.DeviceInfo()
di.SetDeviceClass("BaslerCamEmu")

# you could also set more device filters like e.g.:
# these are combined as AND 
# di.SetSerialNumber("2134234")

<pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x7761923b9b10> >

In [14]:
devs = tlf.EnumerateDevices([di,])
devs

(<pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x7761923b9f00> >,
 <pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x7761f041d930> >,
 <pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x7761f041d1e0> >,
 <pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x7761f041dc60> >,
 <pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x7761f041d3c0> >)

In [15]:
cam_array = py.InstantCameraArray(NUM_CAMERAS)

In [16]:
for idx, cam in enumerate(cam_array):
    cam.Attach(tlf.CreateDevice(devs[idx]))

In [17]:
cam_array.Open()

In [18]:
# store a unique number for each camera to identify the incoming images
for idx, cam in enumerate(cam_array):
    camera_serial = cam.DeviceInfo.GetSerialNumber()
    print(f"set context {idx} for camera {camera_serial}")
    cam.SetCameraContext(idx)

set context 0 for camera 0815-0000
set context 1 for camera 0815-0001
set context 2 for camera 0815-0002
set context 3 for camera 0815-0003
set context 4 for camera 0815-0004


In [19]:
# set the exposure time for each camera
for idx, cam in enumerate(cam_array):
    camera_serial = cam.DeviceInfo.GetSerialNumber()
    print(f"set Exposuretime {idx} for camera {camera_serial}")
    cam.ExposureTimeRaw = 10000

set Exposuretime 0 for camera 0815-0000
set Exposuretime 1 for camera 0815-0001
set Exposuretime 2 for camera 0815-0002
set Exposuretime 3 for camera 0815-0003
set Exposuretime 4 for camera 0815-0004


/tmp/ipykernel_5916/2745882153.py:5: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ExposureTimeRaw.Value = 10000
  cam.ExposureTimeRaw = 10000


In [20]:
# wait for all cameras to grab 10 frames
frames_to_grab = 10
# store last framecount in array
frame_counts = [0]*NUM_CAMERAS


In [21]:
frame_counts

[0, 0, 0, 0, 0]

In [22]:
cam_array.StartGrabbing()
while True:
    with cam_array.RetrieveResult(1000) as res:
        if res.GrabSucceeded():
            img_nr = res.ImageNumber
            cam_id = res.GetCameraContext()
            frame_counts[cam_id] = img_nr
            print(f"cam #{cam_id}  image #{img_nr}")
            
            # do something with the image ....
            
            # check if all cameras have reached 100 images
            if min(frame_counts) >= frames_to_grab:
                print( f"all cameras have acquired {frames_to_grab} frames")
                break
                
                
cam_array.StopGrabbing()

cam #0  image #1
cam #1  image #1
cam #2  image #1
cam #3  image #1
cam #4  image #1
cam #0  image #2
cam #1  image #2
cam #2  image #2
cam #3  image #2
cam #4  image #2
cam #0  image #3
cam #1  image #3
cam #2  image #3
cam #3  image #3
cam #4  image #3
cam #0  image #4
cam #1  image #4
cam #2  image #4
cam #3  image #4
cam #4  image #4
cam #0  image #5
cam #1  image #5
cam #2  image #5
cam #3  image #5
cam #4  image #5
cam #0  image #6
cam #1  image #6
cam #2  image #6
cam #4  image #6
cam #3  image #6
cam #0  image #7
cam #1  image #7
cam #3  image #7
cam #4  image #7
cam #2  image #7
cam #0  image #8
cam #1  image #8
cam #3  image #8
cam #4  image #8
cam #2  image #8
cam #0  image #9
cam #1  image #9
cam #3  image #9
cam #4  image #9
cam #2  image #9
cam #0  image #10
cam #1  image #10
cam #3  image #10
cam #4  image #10
cam #2  image #10
all cameras have acquired 10 frames


In [23]:
cam_array.Close()

In [24]:
frame_counts


[10, 10, 10, 10, 10]